# Implementing dynamic programming for edit distance

In [1]:
def editDistRecursive(x, y):
    # This implementation is very slow
    if len(x) == 0:
        return len(y)
    elif len(y) == 0:
        return len(x)
    else:
        distHor = editDistRecursive(x[:-1], y) + 1
        distVer = editDistRecursive(x, y[:-1]) + 1
        if x[-1] == y[-1]:
            distDiag = editDistRecursive(x[:-1], y[:-1])
        else:
            distDiag = editDistRecursive(x[:-1], y[:-1]) + 1
        return min(distHor, distVer, distDiag)

In [2]:
def editDistance(x, y):
    D = []
    for i in range(len(x) + 1):
        D.append([0] * (len(y) + 1))
        
    for i in range(len(x)):
        D[i][0] = i
    for i in range(len(y) + 1):
        D[0][i] = i
        
    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
                
            D[i][j] = min(distHor, distVer, distDiag)
    
    return D[-1][-1]

In [3]:
%%time
x = 'shake spea'
y = 'Shakespear'
print(editDistRecursive(x,y))

3
CPU times: user 3.82 s, sys: 0 ns, total: 3.82 s
Wall time: 3.85 s


In [4]:
%%time
x = 'shake spea'
y = 'Shakespear'
print(editDistance(x,y))

3
CPU times: user 158 µs, sys: 0 ns, total: 158 µs
Wall time: 152 µs


# Implementing global alignment

In [5]:
alphabet = ['A', 'C', 'G', 'T']

# penatly matrix
score = [[0, 4, 2, 4, 8], \
        [4, 0, 4, 2, 8], \
        [2, 4, 0, 4, 8], \
        [4, 2, 4, 0, 8], \
        [8, 8, 8, 8, 8]] 

In [6]:
def globalAlignment(x, y):
    D = []
    for i in range(len(x) + 1):
        D.append([0] * (len(y) + 1))
        
    for i in range(1, len(x) + 1):
        D[i][0] = D[i-1][0] + score[alphabet.index(x[i-1])][-1]
    for i in range(1, len(y) + 1):
        D[0][i] = D[0][-1] + score[-1][alphabet.index(y[i-1])]
        
    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            distHor = D[i][j-1] + score[-1][alphabet.index(y[j-1])]
            distVer = D[i-1][j] + score[alphabet.index(x[i-1])][-1]
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + score[alphabet.index(x[i-1])][alphabet.index(y[j-1])]
                
            D[i][j] = min(distHor, distVer, distDiag)
    
    return D[-1][-1]

# Overlaps between pairs of reads

In [19]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [8]:
overlap('TTACGT', 'CGTACCGT')

3

In [10]:
overlap('TTACT', 'CGTACCGT')  # only overlap of 2

0

# Finding and representing all overlaps

In [12]:
from itertools import permutations

list(permutations([1,2,3], 3))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [14]:
def naive_overlap_map(reads, k):
    olaps = {}
    for a,b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a,b)] = olen
    return olaps

In [18]:
reads = ['ACGGATC', 'GATCAAGT', 'TTCACGGA']
print(naive_overlap_map(reads, 3))

{('ACGGATC', 'GATCAAGT'): 4, ('TTCACGGA', 'ACGGATC'): 5}
